# HST WFC3/UVIS Reduction and Dark Correction for a Set of Observations

This notebook describes the step-by-step procedure required to improve the reduction of darks and include these in the reduction of a set of Hubble Space Telescope (HST) Wide Field Camera 3 (WFC3)/UV-Visible (UVIS) observations. This is based on the Space Telescope Science Institute (ST/STScI) standard WFC3/UVIS darks reduction pipeline as copied from GitHub 19th April 2019. All changes made to the ST codes should be denoted by "LP" in the comments. Written by Laura Prichard, April–September 2019. Updated September 2020.

<span style="color:blue">NOTE: All improvements to the STScI HST/WFC3/UVIS darks pipeline implemented in this notebook up to Step 23 have now been adopted as standard by the WFC3 Team for future delivered data products. These changes will likely go into effect from ~November 2020, after which the data currently available on MAST will be re-processed to include these changes. Look out for announcements of when these changes will be implemented, and check the headers of the MAST data to verify these changes have been applied to the FLCs that you would like to download. Additional corrections listed from Step 24 onward have not been applied as standard and are advised for the cleanest possible data.</span>

___________________
# Setup

This notebook should come in a darks reduction package (`hst_wfc3_uvis_reduction/`) in the directory `hst_wfc3_uvis_reduction/darks_codes/lp_darks/`. In `darks_codes/` is: 

- `st_darks/` – the ST standard darks reduction copied from GitHub 19th April 2019.
- `lp_darks/` – copy of the st_darks/ directory that was edited.
- `mr_darks/` – copy of Marc Rafelski's codes and reference tables some of which were implemented in the `lp_darks/` version of the ST pipeline.
    
In `darks_codes/lp_darks/` is the following: 

- the CTE correction code directory (`cal_uvis_make_ctecorr_darks/`), 
- the directory of the main codes for the pipeline (`cal_uvis_make_darks/`, main code is `cal_uvis_make_darks.py`), 
- a folder for reference tables i.e. `dark_lookup.txt` (`ref_files/`) that contains an example dark lookup file `dark_lookup_example.txt`,
- an empty folder for terminal outputs for the codes (`logs/`),
- a code to download data with astroquery and organize files ready for reduction (`download_data.py`), 
- a code to CTE correct science data (`ctecorr_scidata.py`),
- codes to check for files and copy/rename them with flexible options (`copy_files.py`),
- code to update the reference files in the file headers (`bestrefs.py`),
- and this notebook (`darks_reduction.ipynb`). 

The darks codes are kept in a root directory `hst_wfc3_uvis_reduction/` along with the data directory e.g., `darks_data/`. This should contain three more directories (all empty), one for the raw data (e.g., `raw_darks/`), reduced data (e.g., `red_darks/`) and a directory for STScI calibration files (e.g., `st_calib/`, more info below on this). These will be in the `hst_wfc3_uvis_reduction/darks_data/` directory, otherwise they should be made should be made in a location of choice but be kept seperate from the code directory, e.g.:

- `darks_data/red_darks/` – location for the reduced data pipeline outputs
- `darks_data/raw_darks/` – location of the raw data
- `darks_data/st_calib/` – location of STScI calibration files used by the pipeline (more info below)

**0) Run functions and packages used in this notebook**

In [ ]:
import os
import glob
import shutil
import numpy as np
import astropy.io.fits as fits
import pandas as pd
from pdb import set_trace as st

# Setting pandas display options
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# Import codes from the directory
import copy_files as cf
import bestrefs as br

**1) Set directories**

They should be defined and made if they don't exist. An example of this is below:

In [ ]:
# Edit and run this to define and make the following directories
import os

# Setting path names
DARK_ROOT = "/user/lprichard/hst_wfc3_uvis_reduction"  #EDIT! Root directory for whole of the darks codes and data
CODE_DIR = os.path.join(DARK_ROOT, 'darks_codes')  #Location of the darks_codes directories that come with this darks reduction package
DAT_DIR = os.path.join(DARK_ROOT, 'darks_data')    #Data directory to contain the following sub directories
RAW_DIR = os.path.join(DAT_DIR, 'raw_darks')       #Raw data directory
RED_DIR = os.path.join(DAT_DIR, 'red_darks')       #Reduced data directory
CAL_DIR = os.path.join(DAT_DIR, 'st_calib')        #ST calibration files directory

print('DARK_ROOT =', DARK_ROOT)
print('CODE_DIR =', CODE_DIR)
print('DAT_DIR =', DAT_DIR)
print('RAW_DIR =', RAW_DIR)
print('RED_DIR =', RED_DIR)
print('CAL_DIR =', CAL_DIR)

# Making data directories if they don't exist
if not os.path.exists(DAT_DIR): 
    os.makedirs(DAT_DIR, 0o774)
if not os.path.exists(RAW_DIR): 
    os.makedirs(RAW_DIR, 0o774)
if not os.path.exists(RED_DIR): 
    os.makedirs(RED_DIR, 0o774)
if not os.path.exists(CAL_DIR): 
    os.makedirs(CAL_DIR, 0o774)

**2) Add the codes directory to your PYTHONPATH**

Add the folder *above* `cal_uvis_make_ctecorr_darks/` and `cal_uvis_make_darks/` to your `PYTHONPATH`, e.g. in `~/.bashrc`

    export PYTHONPATH="[CODE_DIR]/lp_darks:$PYTHONPATH"

Initialize each directory if not done already `DARK_ROOT/darks_codes/lp_darks/cal_uvis_make_ctecorr_darks/` and `DARK_ROOT/darks_codes/lp_darks/cal_uvis_make_darks/`, do `touch __init__.py`

**3) Get STScI standard calibration files**

Copies of the STScI standard calibration files used in the reduction are required. These should be put in the `st_calib/` directory (or another if preferred). Set this location of the calibration files directory as an input to the main darks reduction code (`cal_uvis_make_darks.py`) using the flag `-c|--cal_dir`.
    
Get the latest copy of the calibration files (from Catherine Martlin - cmartlin@stsci.edu, or any of the WFC3 team with access to the files below) and put them in `st_calib/` (or your preferred location):

    /grp/hst/wfc3k/uvis_darks/exclude.list
    /grp/hst/wfc3k/uvis_darks/crr_for_dark.fits
    /grp/hst/wfc3k/uvis_darks/crr_for_hotpix.fits
    /grp/hst/wfc3b/calibration/history.txt
    
According to Catherine, history.txt is the only one that is regularly updated and is done by Sylvia Baggett (WFC3 Branch Manager) around every month with both anneal (publicly available) AND lockups (not public), making this file internal to ST. A new copy of this file and the others (to be sure) are required regularly.

**Set up your reference file (`iref`) directory**

The STScI reference files used in this WFC3 reduction process (`iref`) can be obtained from https://hst-crds.stsci.edu/ and set up using the instructions in this [STScI notebook](https://spacetelescope.github.io/notebooks/notebooks/DrizzlePac/Initialization/Initialization.html) under the Reference Files section. More information [here](http://www.stsci.edu/itt/review/2008_HST_Docs/WFC3_DHB/wfc3_Ch53.html#63144) (use Safari or other browser if not working on Chrome). If running the notebook within the STScI network, the reference files are in `/grp/hst/cdbs/iref`. Point to this directory prior to running code, e.g.:

    export iref="/grp/hst/cdbs/iref/"

____________________
# Download and Organize Darks

**4) Get anneal cycle dates**

Get the dates of the observations in question (e.g. from MAST https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) and then find the WFC3 anneal cycles that span the dates of the observations. From http://www.stsci.edu/hst/instrumentation/wfc3/performance/monitoring/complete-anneal-history (try Safari or other browser if not working with Chrome, email WFC3 team member if not up to date), get a list of the dates & times for all anneal cycles that your span your data, e.g. 

    58561.85943287  2019.078 20:37:35 Mar 19 anneal
    
This is the start date (MJD YEAR.DAY HH:MM:SS, Month YY anneal) of the anneal cycle. The anneal cycle ends the file before that of the next listed anneal date and time, e.g.:

    58590.47559028  2019.107 11:24:51 Apr 17  anneal
    
Get the start/end anneal cycle Modified Julien Date (MJD, first value) for ONE anneal cycle at a time as inputs to the codes below.

If wanting to process darks from the **current** anneal cycle, the anneal start date should still be given, and the anneal end date can be now. This is generated in MJD using the following code:

In [ ]:
from astropy.time import Time

now = Time(Time.now(), format='iso').mjd

print('MJD Now:', now)
print('-----------------------------')

# Or can convert the YEAR.DAY HH:MM:SS format to MJD with the following
in_date = '2019.107 11:24:51'

# Convert the input date string to month/day and MJD
in_date = in_date.replace('.', ':').replace(' ', ':')
print('Input date (year:day:time):', in_date)
out_date = Time(in_date, format='yday').mjd
str_date = Time(out_date, format='mjd').strftime('%d %B %Y, %H:%M:%S')
print('Input date (string): {}'.format(str_date))
print('Input date (MJD): {}'.format(out_date))

**5) Download data with astroquery and organize**

Then run the following code that makes directories, downloads data with astroquery and sorts raw dark files ready for CTE corrections using `hst_wfc3_uvis_reduction/darks_codes/lp_darks/download_data.py`. Run using the following command in `hst_wfc3_uvis_reduction/darks_codes/lp_darks/`:

        python download_data.py [-t|--type] [-s|--data_start] [-e|--data_end] [-p|--proposal_id] [-r|--raw_dir] [-d|--download] [-l|--dload_date]

Required:
    
- `[-t|--type]` – String, the type of data to be downloaded, either ``dark`` for raw darks or ``science`` for raw science data. ``-s --data_start`` and ``-e --data_end`` must be set to the anneal start and end (or present) dates if `--type=='dark'`. ``-p --proposal_id`` must be set if `--type=='science'`, and start and end dates within that proposal ID are optional.
- `[-s|--data_start]` – String/float, required if `--type=='dark'`: MJD value of start date of ONE anneal cycle to at least 6 d.p. from the webpage above (Step 4). Optional if `--type=='science'`: science data start date if a section of a proposal's data is to be downloaded rather than the whole program.
- `[-e|--data_end]` – String/float, required if `--type=='dark'`: MJD value of end date of ONE anneal cycle to at least 6 d.p. from the webpage above (Step 4). Optional if `--type=='science'`: science data end date if a section of a proposal's data is to be downloaded rather than the whole program. All files up to but NOT INCLUDING the end date are selected.
- `[-p|--proposal_id]` – String, proposal ID of the science data to be downloaded, REQUIRED if `--type=='science'`. NOT NEEDED for darks download.
- `[-r|--raw_dir]` – String, path to the directory where raw data should be stored. A directory for each anneal cycle will be made within that, along with seperate sub-folders for downloaded, raw and CTE corrected darks. No trailing slash "/".
    
Optional:

- `[-d|--download]` – Flag, is ``True`` if provided, ``False`` otherwise and is required if you want astroquery to download the raw files.
- `[-l|--dload_date]` – String, download date, should be set if `-d|--download` is not set as this points to the directory where the data were downloaded which is timestamped. Should be in the format ``YYYYMonDD`` e.g. ``2019Aug13``.
    
Execution of this script will create the following file tree:

    <anneal_date:YYYYMMMDD>anneal_rawdarks_aquery<download_date:YYYYMMMDD>/
        ctecorr_darks/
        mastDownload/HST/  (created by the astroquery download command)    
            id**/          (creates ID specific folders with the raw files in)
        raw_darks/         (folder for the raw darks combined from their individual download directories)
    
An example of using the code in a terminal is below:

In [ ]:
# ************************************************************
# Example of inputs
# -t|type = 'dark'
# -s|--anneal_start = '58676.18376157'
# -e|--anneal_end = '58694.25990740'
# -r|--raw_dir = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks'

# Set the following flag which will set download to ``True``
# -d|--download
# OTHERWISE set dload_date to point to a directory of data downloaded 
# on this date to do just the file organization and copying
# -l|--dload_date = '2019Aug13'
# ************************************************************

# Move into codes directory
# cd /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks

# Use the following command in a terminal, the "mode" and "now" definitions ahead 
# are for the log name that the terminal output is piped to which is called at the 
# end of the commnad. "pdb" ipython command is for debugging.

# Set to download
now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run download_data.py  -t 'dark' -s '58676.18376157' -e '58694.25990740' \
    -r '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks' -d" 2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_aquery_$now.txt
# OR for file organization of downloaded data
now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run download_data.py  -t 'dark' -s 58676.18376157 -e 58694.25990740 \
    -r '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks' -l '2019Aug13'" 2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_aquery_$now.txt

**6) Update the file headers with the latest reference files**

Regardless of when the data were downloaded, by running `bestrefs` you can automatically set the current best reference files to use for your data in the headers. This should be done **before** setting any custom header keywords, as done later in the reduction. This is a script given in this code directory that depends on HST's command line tools [`crds.bestrefs`](https://hst-crds.stsci.edu/static/users_guide/command_line_tools.html). The code below prints the existing reference files (`br.print_hdr_keys`) before and after updates. You can see any proposed changes to the headers by setting `br.set_bestrefs` with `update=False`, and when you are happy to update, set `update=True`. 

Input the combined raw data directory that was made in previous step with `download_data.py` (`[raw_dir]/[anneal_dir]/raw_darks`). If for any reason you wish to use different reference files, set these manually. And to get the untouched raws from the download, simply re-run the previous step on the pre-downloaded data to file organize (with the download date `-l` provided and no download `-d` flag set) and recreate this untouched raw data directory.

Set the iref environement if not already set (see Step 3).

In [ ]:
# Run bestrefs on the raw files to update all reference files or specify one keyword to update

# ------------------------------
# INPUTS
# Set the raw data directory set in step 6)
RWD_DIR = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks/2019Mar19anneal_rawdarks_aquery2019Aug13/raw_darks'

# Set update=True to update the headers, or update=False to print the reference files that will be updated.
update = True
# ------------------------------

# For each file in there, check the headers and update:
image_list = glob.glob(os.path.join(RWD_DIR,'*raw.fits'))

# Print existing reference files (or a keyword: e.g., key='BIASFILE') in headers
br.print_hdr_keys(image_list)

# To print reference files and any updates, set update=False, to update headers update=True
br.set_bestrefs(image_list, update=update)

# Print updated reference files (or a keyword: e.g., key='BIASFILE') in headers
br.print_hdr_keys(image_list)

________________________
# CTE Correct Darks

**7) Check WFC3 CTE correction executable**

____________________________
NOTE: The following method is based on an old CTE correction code. As of Feb 2020, STScI are testing a new CTE correction code. Check back to the GitHub repository or the STScI website for updates prior to running this step in case this new code has been released (estimated November 2020). After the release, all MAST data will be reprocessed with this new CTE correction code and the WFC3 team have adopted the adaptations made here to the darks pipeline. Therefore, you should be able to download FLCs from MAST of the same quality as produced with this code and will just need to run the final FLC corrections for the best quality science data, i.e. step 22) onwards.
____________________________

Using the edited version of `cal_uvis_make_ctecorr_darks.py`

    hst_wfc3_uvis_reduction/darks_codes/lp_darks/cal_uvis_make_ctecorr_darks/cal_uvis_make_ctecorr_darks.py
    
This does not depend on Quicklook (as used by STScI) and the files are organized using information in the header (rather than the default STScI file organization). This code is dependent on the standard CTE corrections code written by Jay Anderson and available here:

http://www.stsci.edu/hst/instrumentation/wfc3/software-tools/cte-tools

In a software directory of choice, download the file using `copy link address` on the above web page 
    
    e.g. cd /user/lprichard/software
    wget http://www.stsci.edu/~jayander/X/EXPORT_WFC3UV_CTE/wfc3uv_ctereverse.F
    
Following instructions on the web page, then do the following:

    gfortran wfc3uv_ctereverse.F -o wfc3uv_ctereverse.e
    
WARNING, the code must then be run on the computer that the software is compiled on. The SOFTWARE_DIR is a required input to the code. The webpage also says the following:

- _"This particular routine insists that all the exposures be in the same directory.  The executable can be in a different directory."_

**8) Run CTE correction**

Calling sequence of the CTE correction code:

    python cal_uvis_make_ctecorr_darks.py [-c|--ctecorr_dir] [-r|--rwd_dir] [-s|--software_dir]
        
Required:

- `[-c|--ctecorr_dir]` – String, path to the output CTE corrected data directory (as made in `download_data.py` to `[raw_dir]/[anneal_dir]/ctecorr_darks`, no trailing slash "/".
- `[-r|--rwd_dir]` – String, path to the input raw data directory (all files in one folder as done in previous step with `download_data.py` to `[raw_dir]/[anneal_dir]/raw_darks`), no trailing slash "/".
- `[-s|--software_dir]` – String, path to the compiled CTE correction code `./wfc3uv_ctereverse.e`, no trailing slash "/".
    
The following is an example of running the code in a terminal:    

In [ ]:
# ************************************************************
# Example of inputs
# -c|--ctecorr_dir = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks/2019Mar19anneal_rawdarks_aquery2019Aug13/ctecorr_darks'
# -r|--rwd_dir = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks/2019Mar19anneal_rawdarks_aquery2019Aug13/raw_darks'
# -s|--software_dir = '/user/lprichard/software' 
# ************************************************************

# Move into codes directory and start screen, can take a long time (days) depending on processing power
# cd /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/cal_uvis_make_ctecorr_darks
# screen -S ctecorr

# Use the following command in a terminal, the "mode" and "now" definitions ahead are for the log name 
# that the terminal output is piped to which is called at the end. "pdb" ipython command is for debugging.
now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run cal_uvis_make_ctecorr_darks.py \
    -c '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks/2019Mar19anneal_rawdarks_aquery2019Aug13/ctecorr_darks' \
    -r '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks/2019Mar19anneal_rawdarks_aquery2019Aug13/raw_darks' \
    -s '/user/lprichard/software'" 2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_ctecorr_$now.txt


__________________
# Reduce Darks

**9) Get anneal dates as inputs**

As in step 4), get the dates of the observations in question (e.g. from MAST https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html) and then find the anneal cycles needed for those data from http://www.stsci.edu/hst/instrumentation/wfc3/performance/monitoring/complete-anneal-history (try Safari or other browser if not working with Chrome, email WFC3 team member if not up to date).

Get start and end dates/times for **ONE** anneal cycle at a time in the form ``YYYYMMDD-HH:MM:SS``. An anneal cycle is labeled by its *start* date, i.e. March 19 2019 anneal has format `58561.85943287 2019.078 20:37:35 19-Mar anneal` on the webpage, this translates to an `--anneal_date` input of `20190319-20:37:35`. The appropriate `--endtime` of this anneal cycle is the start date of the next anneal cycle (April 17 2019) as all files *before* this time are included in the anneal cycle. So from the webpage: `58590.47559027 2019.107 11:24:51 17-Apr anneal` tanslates to an `--endtime` input of `20190417-11:24:50`.

Again, if wanting to process darks from the **current** anneal cycle, the anneal start date should still be given, and the anneal end date can be now. This is generated in the right format below. For convenience, you can convert the start and end times to the required input format using the Modified Julien Date (MJD; values from link above) using the following code:

In [ ]:
from astropy.time import Time

# ------------------------------
# INPUT
# Set the input anneal cycle start and end MJD dates
anneal_start = 58561.85943287
anneal_end = 58590.47559027
# ------------------------------

# Converting MJD to YYYYMMDD-HH:MM:SS format
# Anneal start date
t_start = Time(anneal_start, format='mjd')
an_start = t_start.strftime('%Y%m%d-%H:%M:%S')
print('Anneal start date: ', an_start)

# Anneal end date
t_end = Time(anneal_end, format='mjd')
an_end = t_end.strftime('%Y%m%d-%H:%M:%S')
print('Anneal end date: ', an_end)

# Anneal end date of now if in present cycle
now = Time(Time.now(), format='iso').mjd
t_now = Time(now, format='mjd')
an_now = t_now.strftime('%Y%m%d-%H:%M:%S')
print('Now end date: ', an_now)

**If running the code in mode (-m) "prod"**

This mode is currently the default used by STScI (although developments to change this default are underway as of Sep 2020). With this code, it is advised to use the ``dev`` mode which improves the quality of the darks significantly. ``dev`` takes darks from the concurrent anneal cycle to replace good pixels from, rather than from the previous anneal cycle (i.e. ``prod`` mode). `prod` mode was previously used to save time, otherwise you have to wait for the anneal cycle to end before processing data can begin. 

If you wish to use the code using the ``prod`` mode then this is possible but you have to retreive the masterdark from the previous anneal cycle in order to do so. These are stored in the following directory and can be requested from the WFC3 team:

    /grp/hst/wfc3k/uvis_darks/masterdarks/
    
Masterdarks have the following naming structure, where the below file is for e.g., Feb 21, 2019, the start date of the previous anneal cycle to that being run, e.g., March 19, 2019

    masterdark_2019-02-21_ctecorr.fits

These should be saved locally to the `masterdarks/` directory that should be made *within the reduction directory prior to running the code* and this masterdark placed there e.g.,

    /user/lprichard/hst_wfc3_uvis_reduction/darks_data/red_darks/masterdarks
    
NOTE: If running in ``dev`` (advised) mode, the `masterdarks/` directory will be made automatically in the next step.

**10) Run the code**

Calling sequence of the darks reduction code:

    python cal_uvis_make_darks.py [-a|--anneal_date] [-e|--endtime]
        [-c|--ctecorr] [-p|--postflash] [-m|--mode] [-r|--red_dir]
        [-d|--ctecorr_dir] [-l|--cal_dir] [-i|--iref_dir] [-f|--fitpix]
        
Required:

- `[-a|--anneal_date]` – String, start date for ONE anneal in format generated in Step 9 from MJD date.
- `[-e|--endtime]` – String, end date for ONE anneal in format generated in Step 9 from MJD date. OR today's date if within the anneal cycle to be reduced.
- `[-r|--red_dir]` – String, path to reduction directory for outputs and reduced data, no trailing slash "/". A directory for each anneal cycle is made within that.
- `[-d|--ctecorr_dir]` – String, path to raw CTE corrected data directory, no trailing slash "/".
- `[-l|--cal_dir]` – String, path to STScI calibration files needed for reduction (Step 3), no trailing slash "/".

Optional:

- `[-c|--ctecorr]` – Flag, advised with this version. Sets CTE corrected to ``True``, use for data that was CTE corrected prior to running code.
- `[-p|--postflash]` – Flag, advised. Sets postflash to ``True``, postflashed darks will be used.
- `[-m|--mode]` – String, method of replacing pixels in the superdarks, ``dev`` takes pixels from the concurrent anneal cycle's masterdark, ``prod`` takes pixels from the previous anneal's masterdark (see note above for obtaining this). Advised ``dev``, default ``dev``, other option ``prod`` (current ST default, but not post ~Nov 2020).
- `[-i|--iref_dir]` – String, path to IREF files needed for reduction (Step 3). Default ``/grp/hst/cdbs/iref``, set if different. 
- `[-f|--fitpix]` – Flag, advised. Option to create a hot pixel threshold function by row number to find the number of hot pixels to match that close to the read out, i.e. increases completeness of identifying hot pixels. The value is ``True`` (i.e. fit hot pixels with a threshold function) if provided and ``False`` (i.e. don't fit hot pixels with a threshold function) if not provided, in which case a constant hot pixel threshold (0.015 e-/s) will be used (standard in the STScI pipeline, but `fitpix` will be new default from ~Nov 2020).
    
The following is an example of running the code in a terminal:

In [ ]:
# *******************************************************************************
# Example of inputs
# Must be run for one anneal cycle at a time
# -a|--anneal_date = '20190319-20:37:35'
# -e|--endtime = '20190417-11:24:51'
# -m|--mode = 'dev'

# Directories set by user
# -r|--red_dir = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/red_darks'    #Reduction directory for all output files, advised not to put with data
# -d|--ctecorr_dir = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks/2019Mar19anneal_rawdarks_aquery2019Aug13/ctecorr_darks'
# -l|--cal_dir = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/st_calib'
# -i|--iref_dir = '/grp/hst/cdbs/iref'   #same as default OR '/user/lprichard/hst_wfc3_uvis_reduction/lp_darks/myref/'

# Set the following flags which will set each to ``True``
# -c|--ctecorr
# -p|--postflash
# -f|--fitpix    
# *******************************************************************************

# Move into codes directory and start screen, can take several hours depending on processing power
# cd /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/cal_uvis_make_darks
# screen -S darks
        
# Use the following command in a terminal, the "mode" and "now" definitions ahead are for the log name 
# that the terminal output is piped to which is called at the end. "pdb" ipython command is for debugging.
mode="dev" && now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run cal_uvis_make_darks.py -a '20190319-20:37:35' \
    -e '20190417-11:24:51' -c -p -m '$mode' -r '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/red_darks' \
    -d '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/raw_darks/2019Mar19anneal_rawdarks_aquery2019Aug13/ctecorr_darks' \
    -l '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/st_calib' -i '/grp/hst/cdbs/iref' -f" \
    2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_inputtest_$now.txt


__________________
# Copying and Delivering Darks

**11) Copy darks to the combined superdark directory**

The superdarks created in the previous step (`'d*_drk.fits'`)  are rolling 4-day window darks that are used to correct the data. In this step, these are copied to a central `superdarks/` directory for all anneal cycles, from which the best ones to use for your data are selected. These superdarks have been CTE corrected in the previous step, however we take the darks with the file name of `drk.fits` rather than the identical files of a different name `drc.fits` as the naming convention is needed later for running `calwf3` to create reduced science data files without trying to re-run the CTE correction. 

The following code makes a `superdarks/` directory in the reduction directory e.g., `darks_data/red_darks/` for a combined directory of all the superdarks if it doesn't already exist. It then checks for files in the destination and copies the superdarks from a specified anneal cycle directory (made by `cal_uvis_make_darks.py` in the previous step) to that directory if they don't already exist. The anneal cycle directory has the following form: `post-anneal-<anneal_date:YYYYMMDD>_procd-<process_date:YYYYMMDD>_<flags>/`, and should be copied and pasted in below.

In [ ]:
import os
import glob
import shutil

# ------------------------------
# INPUTS
#These should be set by the user
RED_DIR = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/red_darks'  #General reduction directory containing the anneal directories
ANN_DIR = os.path.join(RED_DIR, 'post-anneal-20190319_procd-20190910_ctecorr_dev_fitpix')    #Anneal directory from which superdarks should be copied

# These shouldn't require editing if the above file structure is the same
PDARK_DIR = os.path.join(ANN_DIR, 'masterdark_create')  #This is the standard processed darks directory made by the pipeline
SDARK_DIR = os.path.join(RED_DIR, 'superdarks')         #Location of master superdark directory, this is made is cal_uvis_make_darks.py but checked for and made below if it doesn't exist

# If running on server, print commands to copy files
print_cmd=True
# ------------------------------

# Check for destination directory and files, copies them over if they don't exist and prints the command if specified
cf.copy_files_check(PDARK_DIR, SDARK_DIR, files='d*_drk.fits', print_cmd=print_cmd)

**12) Update the `dark_lookup.txt` file**

For processing your data using these darks, a reference table (`dark_lookup.txt`) is needed to identify which files to use. The following code reads in the current `dark_lookup.txt` file (**create a blank one if it doesn't yet exist**) and updates it with any new superdarks (names and useafter date) in the combined superdarks directory. It saves it to a specified output `dark_lookup.txt` file which can (and usually should) be the same as the input.

The input `dark_lookup.txt` file should have `|` delimiters and contain: superdark name (e.g. `d190320201_drk.fits`), and useafter dates (in long form e.g. `Mar 19 2019 20:37:35`, and MJD `58561.85943287037`) per row. 

Below is code to update the `dark_lookup.txt` file with options to set if the MJD useafter date is not listed in the input file (`calc_mjd=True`), and an overwrite option to overwrite existing filenames in the file (shouldn't need to overwrite usually). To save the output file, set `writef=True`, if `False` the combined output will be printed but the file not updated.

In [ ]:
import os
import glob
from astropy.io.fits import getheader
from pdb import set_trace as st
from astropy.time import Time
from datetime import datetime as dt
import pandas as pd

# ---------------------------
# INPUTS
# Location of the copied superdarks, no trailing "/"
RED_DIR = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/red_darks'
SDARK_DIR = os.path.join(RED_DIR, 'superdarks')
# The location of the input dark_lookup.txt file, should have '|' delimeters
infile = '/user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/ref_files/dark_lookup.txt'
# The location of the output dark_lookup.txt file (can be the same as the input), will have filename, useafter date in long form and in MJD, and '|' delimeters
outfile = '/user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/ref_files/dark_lookup.txt'

# If the infile does not have a useafter MJD column, set this to True to calculate one
calc_mjd = False
# Set overwrite to 'False' if you don't want to replace entries for superdarks already listed in dark_lookup.txt
overwrite = False
# If wanting to write the output file, set to 'True', otherwise the output will be printed but not saved to a file
writef = True
# ---------------------------

# Read in the input dark_lookup.txt
# If no useafter_mjd column exists, make one
if calc_mjd==True:
    df_in = pd.read_csv(infile, names='superdark useafter'.split(), delimiter='|')

    # Convert useafter dates to MJD for old file format
    uain_mjds = []
    for i, ua_in in enumerate(df_in['useafter']):
        date = dt.strptime(ua_in, '%b %d %Y %H:%M:%S')
        iso = date.strftime('%Y-%m-%d %H:%M:%S.%f')
        uain_mjds.append(Time(iso, format='iso').mjd)

    # Add new column to the input file for MJD
    df_in['useafter_mjd'] = uain_mjds
    
# If a useafter_mjd column exists, read in file as is
else:
    df_in = pd.read_csv(infile, names='superdark useafter useafter_mjd'.split(), delimiter='|')
    
# Move to the directory with the copied superdarks
os.chdir(SDARK_DIR)    

# Starting arrays for inputs of new superdakrs
filenames = []
useafters = []
useafter_mjds = []

# For each new superdark, get the filenames, useafter dates from header and calculate an MJD useafter date
n=0
for f in glob.glob('d*_drk.fits'):

    # Check if the superdark is already listed in the dark_lookup.txt file
    if not df_in['superdark'].str.contains(f).any() and (overwrite==False):
        
        print('Adding {} to dark_lookup.txt'.format(f))
        
        # Read in superdark header
        hdr = getheader(f)

        # Converting useafter date to MJD to sort
        useafter = hdr['USEAFTER']
        date = dt.strptime(useafter, '%b %d %Y %H:%M:%S')
        iso = date.strftime('%Y-%m-%d %H:%M:%S.%f')
        mjd = Time(iso, format='iso').mjd

        # Storing values for each superdark
        filenames.append(f)
        useafters.append(useafter)
        useafter_mjds.append(mjd)
        
        n+=1
    else:
        print('{} already listed in dark_lookup.txt, not overwriting entry'.format(f))

# Store new superdark values in a data frame
df_new = pd.DataFrame({})
df_new['superdark'] = filenames
df_new['useafter'] = useafters
df_new['useafter_mjd'] = useafter_mjds

# Combine the old and new dark_lookup data frames
df_out = pd.concat([df_in, df_new])

# Remove duplicates if they exist in the combined data frame by superdark filename
df_out.drop_duplicates(subset ="superdark", keep = "first", inplace = True)

# Sort the combined data frame based on useafter MJD date
df_out = df_out.sort_values(by=['useafter_mjd']).reset_index(drop=True)

print(df_out)
print('Added {} new superdarks to {}'.format(n, outfile))

# Save the new dark_lookup.txt file to the specified output file (can be the same as input) with delimeter '|'
if writef==True:
    print('Writing to {}'.format(outfile))
    df_out.to_csv(outfile, index=False, sep='|', header=False)
else:
    print('File not saved, set writef=True to save output to {}'.format(outfile))

____________
____________
____________
____________
____________

# Download and CTE Correct Science Data

**13) Set up the science data directories**

Outside of the `hst_wfc3_uvis_reduction/` parent directory, set a project specific science data directory. The following sets and makes example directories, e.g.,

In [ ]:
import os

# Setting path names
SCI_ROOT = "/user/lprichard/project"         #EDIT! Root directory for a whole science progam
DAT_DIR = os.path.join(SCI_ROOT, 'sci_data')      #Location of the science data both raw and reduced
RAW_DIR = os.path.join(DAT_DIR, 'raw_sci')        #Raw science data directory
RED_DIR = os.path.join(DAT_DIR, 'red_sci')        #Reduced science data directory

print('SCI_ROOT =', SCI_ROOT)
print('DAT_DIR =', DAT_DIR)
print('RAW_DIR =', RAW_DIR)
print('RED_DIR =', RED_DIR)

# Making data directories if they don't exist
if not os.path.exists(DAT_DIR): 
    os.makedirs(DAT_DIR, 0o774)
if not os.path.exists(RAW_DIR): 
    os.makedirs(RAW_DIR, 0o774)
if not os.path.exists(RED_DIR): 
    os.makedirs(RED_DIR, 0o774)

**14) Download the raw data files with astroquery**

Get the proposal ID for the data (e.g., from MAST https://mast.stsci.edu/portal/Mashup/Clients/Mast/Portal.html). Then use `hst_wfc3_uvis_reduction/darks_codes/lp_darks/download_data.py` to download the science data with [`astroquery`](https://astroquery.readthedocs.io/en/latest/) and do file organization as for Step 5 with the darks, but now with different inputs for the science data. Option to download files by association number (ASN) only, at the end of this step. The code makes directories, downloads data with `astroquery` and sorts raw science data files ready for CTE corrections. Run using the following command in `hst_wfc3_uvis_reduction/darks_codes/lp_darks/`:

        python download_data.py [-t|--type] [-s|--data_start] [-e|--data_end] [-p|--proposal_id] [-r|--raw_dir] [-d|--download] [-l|--dload_date]
        
For the science data, ``--type='science'`` and ``--proposal_id`` should be set. If wanting to select files from part of a proprosal (e.g. a large or ongoing program), you can also set the ``--data_start`` and ``--data_end`` dates along with the required ``--proposal_id``. These must be in MJD, all files up to but not including the end date are selected. Can set today's date as the end date using the code below. **UPDATE April 2020: use `astroquery` to get the dates (example in the first cell below) as data start times are slightly different between MAST and `astroquery`. Files will be missed with an `astroquery` command and start time from the MAST website**.

Execution of this script will create the following file tree:

    PID<proposal_id>_rawdata_aquery<download_date:YYYYMonDD>/
        ctecorr_sci/       (for CTE corrected data)
        mastDownload/HST/  (created by the astroquery download command)    
            id**/          (creates ID specific folders with the files in)
        raw_sci/           (for raw data copied from the download directory into a single directory)
        calwf3_sci/        (for processing science data with new darks with calwf3)

For convenience, the code below produces a readable table of data and times for a project ID (PID) from `astroquery`.

In [ ]:
# Search for the files, ASNs and the start times with astroquery rather than taking them from MAST

from astroquery.mast import Observations
import pandas as pd
import numpy as np
from astropy.time import Time

# ---------------------------------
# INPUTS
# Proposal ID, example UPDATE
PID = '12345'
# ---------------------------------

def mjd_to_str(t):
    """Converts Modified Julien Date (MJD) input (t) 
    to a readable date string (t_str)."""
    t_mjd = Time(t, format='mjd')
    t_str = t_mjd.strftime('%Y-%m-%d %H:%M:%S')
    return t_str

# Select all science observations for the proposal ID 
sciobs = Observations.query_criteria(intentType='science', instrument_name="WFC3/UVIS", proposal_id='{}'.format(PID))
# See available columns in the result
print(sciobs.columns)

# Convert astropy table to a dataframe for manipulation
so_df = pd.DataFrame(np.array(sciobs))

# Get the easily readable date string from the MJD dates
so_df['t_min_str'] = so_df['t_min'].apply(mjd_to_str)
so_df['t_max_str'] = so_df['t_max'].apply(mjd_to_str)

# Print just the ASN numbers (obs_id), start and end times (t_min, t_max) in MJD and string form in descending date order
so_df[['obs_id','t_min','t_min_str','t_max','t_max_str']].sort_values('t_min', ascending=False)

**Selecting a subset of data from a project with input dates**

If wanting to select a subset of data within a PID using input start and end times, use the cell below to get the MJD from entries in the table above. These are slightly rounded up and down to avoid missing entries due to rounding issues is `astroquery`. All files observed within the date window are included.

In [ ]:
from astropy.time import Time
from datetime import datetime as dt
import math

# ---------------------------------
# INPUTS
# Start and end dates copied from the t_min_str and t_max_str columns in the table above
start_date = '2019-03-21 05:07:16'
end_date = '2019-04-10 18:12:42'
# ---------------------------------

def round_up(n, decimals=0): 
    multiplier = 10 ** decimals 
    return math.ceil(n * multiplier) / multiplier

def round_down(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n * multiplier) / multiplier

# Determine the MJD for now
now = Time(Time.now(), format='iso').mjd
print('MJD Now:', now)

# Convert these to MJD
start_mjd = Time(start_date, format='iso').mjd
end_mjd = Time(end_date, format='iso').mjd

# Then round these slightly up and down to avoid rounding issues with astroquery and to get the chosen entries
start_mjd_rnd = round_down(start_mjd, decimals=5)
end_mjd_rnd = round_up(end_mjd, decimals=5)

# Print MJDs to be entered into astroquery
print('Rounded down MJD start: {}'.format(start_mjd_rnd))
print('Rounded up MJD end: {}'.format(end_mjd_rnd))

Check that the input dates will return the desired entries, requires the cell above to be run to get the rounded star and end dates (`start_mjd_rnd`, `end_mjd_rnd`). Note if downloading data in batches based on date, rename the output directories from `download_data.py` ran below to reflect the date window of the data.

In [ ]:
# Print entries selected by input dates to check

from astroquery.mast import Observations
import pandas as pd
import numpy as np
from astropy.time import Time

# ---------------------------------
# INPUTS
# Proposal ID, example UPDATE
PID = '12345'
# ---------------------------------

def mjd_to_str(t):
    """Converts Modified Julien Date (MJD) input (t) 
    to a readable date string (t_str)."""
    t_mjd = Time(t, format='mjd')
    t_str = t_mjd.strftime('%Y-%m-%d %H:%M:%S')
    return t_str

# start_mjd
sciobs = Observations.query_criteria(intentType='science', t_min=[start_mjd_rnd,end_mjd_rnd], instrument_name='WFC3/UVIS', proposal_id='{}'.format(PID))
so_df = pd.DataFrame(np.array(sciobs))

print('{} entries to be downloaded with astroquery'.format(len(so_df)))
# Get the easily readable date string from the MJD dates
so_df['t_min_str'] = so_df['t_min'].apply(mjd_to_str)
so_df['t_max_str'] = so_df['t_max'].apply(mjd_to_str)
# Print just the ASN numbers (obs_id), start and end times (t_min, t_max) in MJD and string form in descending date order
so_df[['obs_id','t_min','t_min_str','t_max','t_max_str']].sort_values('t_min', ascending=False).reset_index()

**Download data**

An example of running `download_data.py` from the command line using sample inputs for science raw files: 

In [ ]:
# ************************************************************
# Example of inputs
# -t|--type = 'science'
# -p|--proposal_id = '12345'
# -r|--raw_dir = '/user/lprichard/project/sci_data/raw_sci'

# Set the following flag which will set download to ``True``
# -d|--download
# OTHERWISE set dload_date to point to a directory of data downloaded on this date to do just the file organization and copying
# -l|--dload_date = '2019Sep13'

# If downloading a subset of data from the project (PID)
# -s|--data_start = '58563.21337'
# -e|--data_end = '58583.75882'
# ************************************************************

# Move into codes directory and start screen
# cd /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks
# screen -S sci_dload

# Use the following command in a terminal, the "now" definition is for the log that the terminal
# output is piped to which is called at the end. "pdb" ipython command is for debugging.
# Set to download
now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run download_data.py  -t 'science' -p '12345'\
    -r '/user/lprichard/project/sci_data/raw_sci' -d" 2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_sciaquery_$now.txt
# OR for file organization of downloaded data
now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run download_data.py  -t 'science' -p '12345'\
    -r '/user/lprichard/project/sci_data/raw_sci' -l '2019Sep13'" 2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_sciaquery_$now.txt

# OR for a specific window within the PID, calculated in cells above
# Rounded down MJD start: 58563.21337
# Rounded up MJD end: 58583.75882
now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run download_data.py  -t 'science' -s '58563.21337' -e '58583.75882' -p '12345'\
    -r '/user/lprichard/project/sci_data/raw_sci'  -d" 2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_sciaquery_$now.txt


**ALTERNATIVE METHOD**

If instead of wanting to download data using PID or start and end date, you can download and organize the data using the association number (ASN, get this from MAST) using the following code and the ASNs as inputs. Note, this creates the directories also made in `download_data.py` that will be used later.

In [ ]:
import os
import glob
import shutil
from astroquery.mast import Observations

# ---------------------------------
# INPUTS
# Set a directory within your project dir to download to and organize files within
SCI_DIR = '/user/lprichard/project/sci_data/my_asns'

# List ASNs to download, example UPDATE
ASNs = ['IDXF58020', 'IDXF48020', 'IDXF47020', 'IDXF30020']

# Turn download or copy on and off
download=True
copy=True
# ---------------------------------

def copy_dload_data(DLD_DIR, RWD_DIR):
    """Copies the downloaded files out of the nested astroquery directories 
    and into one combined directory (raw downloaded data, RWD_DIR). From here, 
    the CTE correction code copies over only the darks with the right exposure 
    times to be CTE corrected."""
    # Move to the download directory
    os.chdir(DLD_DIR)

    # Copies data out of the astroquery sub-directories and into the RWD_DIR directory if empty
    print('Copying downloaded raws from astroquery subdirectories {}'.format(DLD_DIR))
    n=0
    
    for i, idob in enumerate(glob.glob('*')):
        # Check if the file is already in the directory, if not it is copied
        src = os.path.join(DLD_DIR, idob, idob +'_raw.fits')
        dst = os.path.join(RWD_DIR, idob +'_raw.fits')
        if not os.path.exists(dst):
            print('Copying {} to {}'.format(src, dst))
            shutil.copy(src, dst)
            n+=1
        else: 
            print('File exists {}, not copying file from {}'.format(dst, DLD_DIR))

    print('Copied {} files to combined raw data directory {}'.format(n, RWD_DIR))
        

# Set sub directories to be made
RWD_DIR = os.path.join(SCI_DIR, 'raw_sci')
CTECORR_DIR = os.path.join(SCI_DIR, 'ctecorr_sci')
        
# Make the raw data directory if it doesn't exist
if os.path.exists(RWD_DIR):
    print('RWD_DIR directory exists:', RWD_DIR)
else:
    os.makedirs(RWD_DIR, 0o774)
    print('Created RWD_DIR directory:', RWD_DIR)
# Make the raw data directory if it doesn't exist
if os.path.exists(CTECORR_DIR):
    print('CTECORR_DIR directory exists:', CTECORR_DIR)
else:
    os.makedirs(CTECORR_DIR, 0o774)
    print('Created CTECORR_DIR directory:', CTECORR_DIR)
        
# For each ASN, make an ASN directory in the SCI_DIR and download data
for ASN in ASNs: 
    
    print('=================================')
    # Set ASN directory
    ASN_DIR = os.path.join(SCI_DIR, ASN)
    
    # Check for the ASN directory and make if it doesn't exist
    if os.path.exists(ASN_DIR):
        print('ASN directory exists:', ASN_DIR)
    else:
        os.makedirs(ASN_DIR, 0o774)
        print('Created ASN directory:', ASN_DIR)

    # Move to ASN directory
    os.chdir(ASN_DIR)

    # Search for files with astroquery for each ASN ID 
    sciobs = Observations.query_criteria(intentType='science', instrument_name="WFC3/UVIS", obs_id='{}'.format(ASN.lower()))
    sciprod = Observations.get_product_list(sciobs)
    rawsci = Observations.filter_products(sciprod, productSubGroupDescription="RAW")
    print('{} raw files found: {}'.format(len(rawsci), rawsci['productFilename']))
    
    # If download option is set, move to the download directory, check if empty, then download
    DLD_DIR = os.path.join(ASN_DIR, 'mastDownload', 'HST')
    if download==True:
        if os.path.exists(DLD_DIR):
            print('Download directory exists!! Not downloading files')
        else: 
            Observations.download_products(rawsci, mrp_only=False)  
            print('Download to {} complete'.format(DLD_DIR))
    
    # Copy files out of the ASN directories into one raw download directory
    if copy==True:
        copy_dload_data(DLD_DIR, RWD_DIR)

**15) Update the file headers with the latest reference files**

As for the darks in Step 6, update the reference files in the headers of the science data. Input the combined raw data directory that was made in previous step with `download_data.py` (`[raw_dir]/[anneal_dir]/raw_darks`).

In [ ]:
# Run bestrefs on the raw files to update all reference files or specify one keyword to update

# ------------------------------
# INPUTS
# Set the raw data directory, as set in step 14)
RWD_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/raw_sci'

# Set update=True to update the headers, or update=False to print the reference files that will be updated.
update = True
# ------------------------------

# For each file in there, check the headers and update:
image_list = glob.glob(os.path.join(RWD_DIR,'*raw.fits'))

# Print existing reference files (or a keyword: e.g., key='BIASFILE') in headers
br.print_hdr_keys(image_list)

# To print reference files and any updates, set update=False, to update headers update=True
br.set_bestrefs(image_list, update=update)

# Print updated reference files (or a keyword: e.g., key='BIASFILE') in headers
br.print_hdr_keys(image_list)

**16) CTE correct the science data**

____________________________
NOTE: The following method is based on an old CTE correction code. As of Feb 2020, STScI are testing a new CTE correction code. Check back to the GitHub repository or the STScI website for updates prior to running this step in case this new code has been released (estimated November 2020). After the release, all MAST data will be reprocessed with this new CTE correction code and the WFC3 team have adopted the adaptations made here to the darks pipeline. Therefore, you should be able to download FLCs from MAST of the same quality as produced with this code and will just need to run the final FLC corrections for the best quality science data, i.e. step 22 onwards.
____________________________

Then CTE correct the science data using `darks_red_pack/darks_codes/lp_darks/ctecorr_scidata.py` which is an adapted version of `cal_uvis_make_ctecorr_darks.py` that checks, copies and CTE corrects the science data. It is called using the following command:

    python ctecorr_scidata.py [-c|--ctecorr_dir] [-r|--rwd_dir] [-s|--software_dir]

Required:

- `-c|--ctecorr_dir` – String, path to output CTE corrected data directory made in `download_data.py`, no trailing slash "/".
- `-r|--rwd_dir` – String, path to raw data directory (all files in one folder as done in previous step with `download_data.py`), no trailing slash "/".
- `-s|--software_dir` – String, path to the compiled CTE correction code `./wfc3uv_ctereverse.e`, no trailing slash "/".

An example of running `ctecorr_scidata.py` from the command line using sample inputs: 

In [ ]:
# *******************************************************************************
# Example of inputs
# -c|--ctecorr_dir = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/ctecorr_sci'
# -r|--rwd_dir = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/raw_sci'
# -s|--software_dir = '/user/lprichard/software' 
# *******************************************************************************

# Move into codes directory and start screen
# cd /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks
# screen -S ctesci

# Use the following command in a terminal, the "now" definition ahead is for the log that the 
# terminal output is piped to which is called at the end. "pdb" ipython command is for debugging.
now=$(date +"%m_%d_%Y") && ipython --pdb -c "%run ctecorr_scidata.py  -c '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/ctecorr_sci'\
    -r '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/raw_sci' \
    -s '/user/lprichard/software'" 2>&1 | tee /user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/logs/log_ctecorrPID12345_$now.txt

____________
# Process Science Data with New Darks

**17) Select which superdarks to use for the science data**

For the CTE corrected science data files, the following code cross-references the superdarks listed in `dark_lookup.txt` that are within the `superdarks/` directory to find which superdarks to use to process the data. It creates two files within the CTE correction science data directory:

- `file_summary.txt` – a list of the science data raw file names, date of the observations, exposure start time in MJD, and the superdark to use for each raw science file.
- `superdarks.txt` – a comma separated version of dark_lookup.txt at the time of running the below code.
    
If not all science data files have a corresponding superdark, a warning is given and the output files are **not saved**. You must therefore make sure you have the relevant superdarks for the science data, that these have been moved to the `superdarks/` directory, and that the `dark_lookup.txt` has been updated to reflect the `superdarks/` directory. You must have the saved `file_summary.txt` file to proceed to the next steps.

In [ ]:
# Written by Ben Sunquist, adapted by Laura Prichard
# see what dark to use in calwf3 for each file

from astropy.time import Time
from datetime import datetime as dt
import astropy.io.fits as fits
import numpy as np
from pdb import set_trace as st
import os
import glob
import pandas as pd

# ----------------------------------
# INPUT
CTECORR_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/ctecorr_sci'
DARK_LOOKUP = '/user/lprichard/hst_wfc3_uvis_reduction/darks_codes/lp_darks/ref_files/dark_lookup.txt'
# ----------------------------------

# get info for rac files
files = glob.glob(os.path.join(CTECORR_DIR, '*rac.fits'))
times = []
expstarts = []
filenames = []
files_raw = []
for f in files:
    filenames.append(os.path.basename(f))
    files_raw.append(os.path.basename(f).replace('rac','raw'))
    info = '{} {}'.format(fits.getheader(f,0)['DATE-OBS'], fits.getheader(f,0)['TIME-OBS'])
    times.append(info)
    expstarts.append(fits.getheader(f,0)['EXPSTART'])

# Save rac file info to data frame
df = pd.DataFrame({})
df['file'] = filenames
df['file_raw'] = files_raw
df['date'] = times
df['expstart'] = expstarts
df = df.sort_values(by=['expstart']).reset_index(drop=True)

# get info for superdarks
df_sd = pd.read_csv(DARK_LOOKUP, names='superdark useafter useafter_mjd'.split(), delimiter='|')
df_sd = df_sd.sort_values(by=['useafter_mjd']).reset_index(drop=True)

# add superdark to use to df for each file
sds = []
t = np.array(df_sd['useafter_mjd'])
n=0
for i in range(len(df)):
    expstart = float(df['expstart'][i])
    if len(np.where(expstart>t)[0])>0:
        sd_match = df_sd['superdark'].iloc[np.where(expstart>t)[0][-1]]
        n+=1
    else:
        sd_match = ''
        print('There is no superdark for science data file {} with start time {}'.format(df['file'][i], df['expstart'][i]))
    sds.append(sd_match)

df['superdark'] = sds

# Check if all the raw files have a superdark, if so saving the output files, if not print warning
if n==len(df):
    print('All raw science data files were matched to a superdark, output saved')
    df.to_csv(os.path.join(CTECORR_DIR, 'file_summary.txt'), index=False)
    df_sd.to_csv(os.path.join(CTECORR_DIR, 'superdarks.txt'), index=False)
else: 
    print('WARNING: There are not superdarks for all of the raw science data, output not saved')

**18) Move science data to processing directory**

The processing directory is where all the science data and the relevant superdarks will be moved to for reduction with `calwf3` to produce `*_flc.fits` files. This process also involves renaming files and updating header information so that calwf3 can run normally on them without factoring in the previous reduction stages that have been changed from the standard procedure (e.g. doesn't try to CTE correct the already corrected files).

The code below moves and renames the `*rac.fits` files to `*raw.fits` files from the user-defined CTE correction directory into the user-defined processing directory if the file doesn't already exist. These directories are both within the same proposal ID directory and are made in Step 14 by `download_data.py`.

In [ ]:
# Move the science data *rac.fits files into a new directory, then rename to *raw.fits so that calwf3 can run on them
# ----------------------------------
# INPUT
CTECORR_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/ctecorr_sci'
PROC_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci'

# Prints command to run it from the terminal, set to False to copy files from the notebook
print_cmd=True
# ----------------------------------

# Makes destination directory if it doesn't exist, checks if files exist, copies them if not, and renames files as specified
cf.copy_files_check(CTECORR_DIR, PROC_DIR, files='*rac.fits', rename=True, src_str='rac', dst_str='raw', print_cmd=print_cmd)

**19) Move superdarks to processing directory**

Using information from the `file_summary.txt` file produced in Step 17, copy the superdarks necessary to process the science data into the processing directory if they don't already exist there.

In [ ]:
import os
import glob
import pandas as pd

# ----------------------------------
# INPUT
CTECORR_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/ctecorr_sci'
PROC_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci'

RED_DIR = '/user/lprichard/hst_wfc3_uvis_reduction/darks_data/red_darks'
SDARK_DIR = os.path.join(RED_DIR, 'superdarks')

# Prints command to run it from the terminal, set to False to copy files from the notebook
print_cmd=True
# ----------------------------------

# Read in the file made previously for the data in the processing directory
df = pd.read_csv(os.path.join(CTECORR_DIR,'file_summary.txt'))

# Check for destination directory and files, copies them over if they don't exist and prints the command if specified
cf.copy_files_check(SDARK_DIR, PROC_DIR, files=df['superdark'].unique(), print_cmd=print_cmd)

**20) Update the headers of the science data and darks to run with calwf3**

Update the headers of the science data with custom calibration files. For each CTE corrected science file (that has been renamed to `raw` from `rac`), replace the dark file to use in the header to the relevant new superdark based on the output `file_summary.txt` file prodcued in Step 17. Also turn the CTE correction flag to `OMIT` as this has already been done separately. 

Update header info of superdarks so they can run with `calwf3`. For the copied superdarks in the processing directory, change the `FILETYPE` key word to `DARK`.

In [ ]:
# Written by Ben Sunquist, adapted by Laura Prichard
# Update the headers of the science data to point to the right superdarks and run with calwf3

import os
import glob
import pandas as pd
import astropy.io.fits as fits

# ----------------------------------
# INPUT
CTECORR_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/ctecorr_sci'
PROC_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci'
# ----------------------------------

# Update science data headers
# update DARKFILE headers in the rac files to the new superdarksand turn off CTE (THESE ARE ALREADY RACS, I.E. CTE CORRECTED)
df = pd.read_csv(os.path.join(CTECORR_DIR, 'file_summary.txt'))

files = glob.glob(os.path.join(PROC_DIR, '*raw.fits'))
n=0
for f in files:
    h = fits.open(f)
    sd = df['superdark'][df['file_raw']==os.path.basename(f)].values[0]
    h[0].header['DARKFILE'] = sd
    h[0].header['PCTECORR'] = 'OMIT'
    h.writeto(f, overwrite=True)
    h.close()
    n+=1
    print('Updated header for {}/{} raws {}'.format(n, len(files), os.path.basename(f)))
    
print('Updated headers for {} raws ready for processing by calwf3 in {}'.format(n, PROC_DIR))

# Update superdark headers
# change superdark filetype to 'DARK' (so calwf3 will run)
dfiles = glob.glob(os.path.join(PROC_DIR,'d*_drk.fits'))
nd=0
for d in dfiles:
    hd = fits.open(d)
    hd[0].header['FILETYPE'] = 'DARK'
    hd.writeto(d, overwrite=True)
    hd.close()
    nd+=1
    print('Updated header for {}/{} superdarks {}'.format(nd, len(dfiles), d))
    
print('Updated headers for {} superdarks ready for processing by calwf3 in {}'.format(nd, PROC_DIR))

**21) Run calwf3 on data**

Within the processing directory, run `calwf3` from the terminal on the updated files using the following commands. This code will produce reduced data files that are ready for drizzling with [DrizzlePac](https://www.stsci.edu/scientific-community/software/drizzlepac.html). 

In [ ]:
# From the terminal using the following example commands

# Move into the processing directory (PROC_DIR) with the science data and renamed darks with updated headers
cd /user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci

# Set the iref environement if not already set
export iref="/grp/hst/cdbs/iref/"

# Run calwf3 on each raw science file
ls *raw.fits | awk '{print "calwf3.e",$1}' | csh

_________________________
<span style="color:green">**If applying additional FLC corrections, skip to Step 24**</span>

**22) Rename and move files**

If applying additional FLC corrections below, skip this renaming step as this will be done later. 

The files produced by `calwf3` will be labeled `*flt.fits` files as it doesn't know that the files have already been CTE corrected. Move these files to a `final/` directory and correctly change their name to `flc.fits` files. These science files are ready to be drizzled together with AstroDrizzle.

In [ ]:
# ----------------------------------
# INPUT
# Set the processing directory and the final directory
PROC_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci'
FIN_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/final'

# Print command for terminal?
print_cmd=True
# ----------------------------------

cf.copy_files_check(PROC_DIR, FIN_DIR, files='*flt.fits', rename=True, src_str='flt', dst_str='flc', print_cmd=print_cmd)

**23) Update headers of files**

Update headers with the final filename and date of creation.

In [ ]:
import os
import glob
from astropy.io import fits
from astropy.time import Time

# ----------------------------------
# INPUT
# For 24 June 2020 data, ran 16 July 2020
FIN_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/final'
# ----------------------------------

# Move into directory
os.chdir(FIN_DIR)

# Set time now
now = Time(Time.now(), format='iso')

# List flcs
flcs = glob.glob('*flc.fits')

# update filenames, history, date
n=0
for f in flcs:
    print('filename: {}'.format(f))

    # Open file for editing
    h = fits.open(f)

    # Update header
    h[0].header['FILENAME'] = '{}'.format(f)
    h[0].header['HISTORY'] = 'FLC corrections by L Prichard performed {}'.format(now)
    h[0].header['HISTORY'] = 'Code: https://github.com/lprichard/hst_wfc3_uvis_reduction.git'

    # Save file
    h.writeto(f, overwrite=True)
    h.close()
    n+=1
    print('Updated header for {}/{}: {}'.format(n, len(flcs), f))

print('Updated headers for {} final flcs in {}'.format(n, FIN_DIR))

<span style="color:blue">NOTE: All improvements to the STScI HST/WFC3/UVIS darks pipeline implemented in this notebook up to Step 23 have now been adopted as standard by the WFC3 Team for future delivered data products. These changes will likely go into effect from ~November 2020, after which the data currently available on MAST will be re-processed to include these changes. Look out for announcements of when these changes will be implemented, and check the headers of the MAST data to verify these changes have been applied to the FLCs that you would like to download. Additional corrections listed from Step 24 onward have not been applied as standard and are advised for the cleanest possible data.</span>

_______________________________________________
_______________________________________________
_______________________________________________
# Additional FLC Corrections

The following are optional but advised for the cleanest possible data. 

**24) Correct for amp offsets (produces smoother images)**

This step subtracts the median background from each of the amps (i.e. image quadrants) in the `*flt.fits` files produced by `calwf3` resulting in smoother final and Drizzled images. The median subtracted `*flt.fits` files produced by this step will be renamed `*flt_medsub.fits`. Go to the Ben Sunnquists's [`make_uvis_skydark.py`](https://github.com/bsunnquist/uvis-skydarks/blob/master/make_uvis_skydark.py) GitHub page and download the latest version to the processing directory (PROC_DIR), e.g.:

    cd /user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci
    wget https://github.com/bsunnquist/uvis-skydarks/blob/master/make_uvis_skydark.py
    
Run the code from the terminal using the example below. Use the `--no_multiply_flat` flag for a simple median subtraction, or remove for a mediam amp value mutiplied by the flat to be subtracted (depends on data type if this is preferred). 

In [ ]:
# Move into PROC_DIR
cd /user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci

# Set iref/ environement if not already set
export iref="/grp/hst/cdbs/iref/"

# To perform median amp subtraction 
ipython --pdb -c "%run make_uvis_skydark.py"

If only applying this additional FLC correction, rename the files from `flt_medsub.fits` to `flc.fits` files, copy them to the final directory, and update their headers as in steps 22 & 23. Otherwise, continue with the other additional correction.

# Apply Read Out Cosmic Ray (ROCR) Corrections

<span style="color:red">**NOTE: the following correction should only be applied to files that have been CTE corrected with the new "dynamic" CTE code, scheduled for release in ~November 2020.**</span> 

Read out cosmic rays (ROCRs) are cosmic rays that fall on the detector whilst the array is being read out. This causes the ROCRs to be over-corrected for by the CTE code, as they actually fell on the detector at a closer distance to the readout than observed, resulting in divots in the images. These overcorrected holes in the data are flagged prior to drizzling so they do not contribute to the final mosaics. The method outlined here is based on the codes written by Ben Sunnquist and available on his GitHub [here](https://github.com/bsunnquist/uvis-skydarks/blob/master/flag_rocrs.ipynb).


**25) Renaming and moving files for further corrections**

From the `median*flat`-amp subtracted files (`*flt_medsub.fits`) in the processing directory above (now listed as `MSF_DIR`), clean copies of the files(`ROCR_DIR`), and a copy to be edited to get the ROCR flags, first with a WCS update (`WCS_DIR`), are renamed and made.

In [ ]:
# ----------------------------------
# INPUT
# Set directories
MSF_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/calwf3_sci'
WCS_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/wcs_updt'
ROCR_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/rocr_clean'

# Running on server? prints command
print_cmd=True
# # ---------------------------

# Makes destination directory if it doesn't exist, checks if files exist, copies them if not, and renames files as specified
cmd1 = cf.copy_files_check(MSF_DIR, WCS_DIR, files='*flt_medsub.fits', rename=True, src_str='flt_medsub', dst_str='flc', print_cmd=print_cmd)
cmd2 = cf.copy_files_check(MSF_DIR, ROCR_DIR, files='*flt_medsub.fits', rename=True, src_str='flt_medsub', dst_str='flc', print_cmd=print_cmd)
if print_cmd==True: print('\n{} && {}'.format(cmd1, cmd2))

**26) Update the WCS of one set of the FLCs**

In order to flag the ROCRs, the FLCs are run through Astrodrizzle. For this, the WCS is updated in advance to prevent Astrodrizzle from crashing. This copy of the files will not be the final output, the flagged ROCRs are copied to the untouched FLCs in `ROCR_DIR`.

In [ ]:
import os
import glob
from stwcs import updatewcs

# ----------------------------------
# INPUT
WCS_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/wcs_updt'
# ----------------------------------

print(WCS_DIR)
os.chdir(WCS_DIR)

# List flcs
files = sorted(glob.glob('*_flc.fits'))

# Update WCS in header to avoid errors
updatewcs.updatewcs(files, use_db=False)    #Set use_db=False with drizzlepac 3.1.6 and above

**27) Copy the files**

Copy the updated WCS files to a new directory for running through astrodrizzle.

In [ ]:
# ----------------------------------
# INPUT
WCS_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/wcs_updt'
DRIZ_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/rocr_driz'
print_cmd=True
# ----------------------------------

# Makes destination directory if it doesn't exist, checks if files exist, copies them if not
cmd = cf.copy_files_check(WCS_DIR, DRIZ_DIR, files='*flc.fits', print_cmd=print_cmd)

**28) Drizzle the FLCs to get CR flags**

The ROCR corrections below rely on having cosmic ray (CR) flags (DQ value 4096), so run them through astrodrizzle to get them if they dont already exist. The files are run with minimal settings and in batches of association number to prevent large mosaics being made.

If flagging of ROCRs to too aggressive or too low (see parameters below), play with the CR flagging S/N thresholds in astrodrizzle (`driz_cr_snr='5.0 4.0'`).

In [ ]:
# Get batches of files to run through astrodrizzle based on ASN ID
import os
import glob
from stsci.tools import teal
from astropy.io import fits
import datetime
from platform import python_version
from drizzlepac import astrodrizzle as ad

# ----------------------------------
# INPUT
DRIZ_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/rocr_driz'
# ----------------------------------

# Move into the drizzle directory
print(DRIZ_DIR)
os.chdir(DRIZ_DIR)

# Then run just the following:
# List flcs
files = sorted(glob.glob('*_flc.fits'))

# Loop to get batches over all files to run through astrodrizzle based on ASN ID
fields = []
asns_full = []
asns = []
for f in files:

    #Get field and ASN ID from fits header
    field = fits.getval(f, 'TARGNAME')
    asn = fits.getval(f, 'ASN_ID')

    # For each ASN ID, store the full ASN ID and file abreviation, and fields/targetnames of observation
    if asn not in asns_full:
        asns_full.append(asn)
        asns.append(f[0:6])
        fields.append(field)  

print('Unique ASNs: {}'.format(asns_full))
print('Unique ASN filenames: {}'.format(asns))
print('Fields: {}'.format(fields))

# Create lists of files associated with each ASN ID:
lists = []
for asn in asns:
    asn_files = [files[i] for i, s in enumerate(files) if asn in s]
    lists.append(asn_files)

# Lists of files for each ASN ID that will be ran by astrodrizzle in batches
print(lists)

teal.unlearn('astrodrizzle')
print('Python version {}'.format(python_version()))
ad.__version__

# Timestamp for drizzles
now = datetime.datetime.now()
print('*****************************************************************************')
print(DRIZ_DIR)
print('LP log: drizzle started at ', now.strftime("%Y-%m-%d %H:%M"))
print('*****************************************************************************')

# Run astrodrizzle with lists and ASN IDs defined above
for l, asn in zip(lists, asns):
    ad.AstroDrizzle(l, 
        driz_cr_corr=True, 
        driz_combine=True,
        preserve=False,  
        clean=True, 
        build=True, 
        driz_cr_snr='5.0 4.0',    #Ben's original params: 3.5 3.0'
        output='{}'.format(asn))

# Timestamp for drizzles
now = datetime.datetime.now()
print('*****************************************************************************')
print(DRIZ_DIR)
print('LP log: drizzle complete at ', now.strftime("%Y-%m-%d %H:%M"))
print('*****************************************************************************')

**29) Make a pre-ROCR copy of the drizzled FLCs**

Make a pre-ROCR copy of the drizzled files in case you need to change any ROCR-flagging parameters to adjust the number of pixels flagged.

In [ ]:
# ----------------------------------
# INPUT
src_dir = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/rocr_driz'
dst_dir = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/PRErocr_driz'

print_cmd=True
# ----------------------------------

# Makes destination directory if it doesn't exist, checks if files exist, copies them if not
cmd = cf.copy_files_check(src_dir, dst_dir, files='*flc.fits', print_cmd=print_cmd)

**30) Correct the ROCRs**

The ROCRs are flagged in the DQ arrays of drizzled FLCs and just the DQ arrays are copied to the untouched/undrizzled FLCs that are then renamed as `*_rocr_flagged_flc.fits` to distinguished from the untouched inputs. To re-run this after changing flagging parameters, delete all `*_rocr_flagged_flc.fits` files from the clean `ROCR_DIR` directory.

Notes from Ben: _"[The code] typically flagged an additional ~ 5000-50,000 pixels in each chip (this # gets printed to the screen when you run the code). If you find it's over/under flagging... you could raise the sigma in the code, or set the driz_cr_snr ~'5 4' rather than the default '3.5 3' when making the CR maps, which sometimes overflags CRs (and thus can overflag ROCRs as well). To verify [the ouput], I blink the FLC SCI (ext=1/4) and DQ (ext=3/6) extensions, and make sure those negative tails attached to some CRs are flagged"_

**Run in astroconda Python 3.6 environments, some incompatibilities with `astropy v4.0`**

In [ ]:
# Flag pixels as "bad detector pixel" (dq value 4) that are within 5 pixels of a CR hit (away from the
# readout direction) AND X sigma below the image mean (where the sigma and mean here are from a Gaussian 
# fit to the sigma-clipped image data).
#
# These pixels are read out cosmic rays (ROCRs), i.e. pixels that fall during readout and therefore 
# trick the CTE correction into overcorrecting them since it thinks they fell farther from the amp
# than they actually did.

import glob
import os
from astropy.io import fits
from astropy.stats import sigma_clip
from astropy.modeling import models, fitting
import numpy as np

###################################### USER INPUTS ######################################
# ----------------------------------
# INPUT
DRIZ_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/rocr_driz'
ROCR_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/rocr_clean'
# ----------------------------------

# LP added
os.chdir(DRIZ_DIR)

# the files to use to find the ROCRs (i.e. drizzling has been done on these files so they have CR flags)
files = sorted(glob.glob('*_flc.fits'))  # the files to flag ROCRs in

# the directory containing the files to add the ROCR flags to (no drizzling has been done on these files)
untouched_files_dir = ROCR_DIR

# the sigma to use when determining the threshold for flagging ROCRs
sigma = 2.75
#########################################################################################

for f in files:
    basename = os.path.basename(f)
    print('Flagging ROCRs in {} ...'.format(basename))
    untouched_file = os.path.join(untouched_files_dir, basename)
    h = fits.open(f)
    h_untouched = fits.open(untouched_file)

    for ext in [1,4]:
        data = h[ext].data
        dq = h[ext+2].data
        dq_untouched = h_untouched[ext+2].data

        # Find lower limit for flaggincg ROCRs
        clipped = sigma_clip(data, sigma=3, maxiters=5)
        d = clipped[clipped.mask==False].data
        n, bins = np.histogram(d, bins=70)
        bin_centers = (bins[:-1] + bins[1:]) / 2
        #LP added [0:1] indexes
        g_init = models.Gaussian1D(amplitude=np.array(n[n==max(n)][0:1]), mean=np.array(bin_centers[n==max(n)][0:1]), stddev=np.std(d))
        
        fit_g = fitting.LevMarLSQFitter()
        g = fit_g(g_init, bin_centers, n)
        
        #LP removed [0] index from g.mean.value
        thresh = g.mean.value - sigma*g.stddev.value
        print('\t Threshold Ext {} = {:.3f} - {}*{:.3f} = {:.3f}'.format(ext, g.mean.value, sigma, g.stddev.value, thresh)) 
        
        # Make mask of all CR hits
        cr_mask = np.zeros(dq.shape, dtype=int)
        cr_mask[dq&4096!=0] = 1

        # Flag pixels within 5 pixels of a CR hit (away from readout) that are below the threshold 
        coords = np.where(cr_mask==1)
        cr_mask_new = np.zeros(cr_mask.shape)
        for i in np.arange(len(coords[0])):
            x,y = coords[1][i], coords[0][i]

            # Get the first y-coordinate to check
            if ext==1:
                running_y = y + 1
            elif ext==4:
                running_y = y - 1
            else:
                print('extension {} not expected'.format(ext))

            # See if this coordinate has a value below the threshold
            count = 0
            while count < 5:  # stay within 5 pixels of cr hit
                if (running_y <= 2050) & (running_y >= 0):  # avoid going off the image y-dimension          
                    val = data[running_y, x]
                    if val < thresh:
                        cr_mask_new[running_y, x] = 1
                    if ext==1:
                        running_y += 1
                    elif ext==4:
                        running_y -= 1
                    else:
                        print('extension {} not expected'.format(ext))
                else:
                    pass

                count += 1

        # Add in new ROCR flags as 4 (bad detector pixel)
        dq_untouched[(dq_untouched&4==0) & (cr_mask_new==1)] += 4
        h_untouched[ext+2].data = dq_untouched

        # Write out ROCR flag map
        fits.writeto(f.replace('_flc.fits','_rocr_map_ext_{}.fits'.format(ext)), cr_mask_new, overwrite=True)
        print('\t # of ROCR flags in Ext {}: {}'.format(ext, len(cr_mask_new[cr_mask_new==1])))

    # Write out the ROCR-flagged flc file
    outname = untouched_file.replace('_flc.fits','_rocr_flagged_flc.fits')
    h_untouched.writeto(outname, overwrite=False)
    h_untouched.close()
    print('\t ROCR-flagged image saved to {}'.format(os.path.basename(outname)))

**31) Move, rename files and update the headers** 

As in skipped steps 22 & 23 if applying these additional corrections, now rename the final files, copy to the final directory, and update their headers to reflect these additional edits.

In [ ]:
# Move the files to the final directory and rename

import glob
import os
import shutil

# ----------------------------------
# INPUT
ROCR_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/rocr_clean'
FIN_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/final'

# Running from terminal? prints command
print_cmd=True
# ----------------------------------

# Makes destination directory if it doesn't exist, checks if files exist, copies them if not, and renames files as specified
cf.copy_files_check(ROCR_DIR, FIN_DIR, files='*_rocr_flagged_flc.fits', rename=True, src_str='_rocr_flagged_flc', dst_str='_flc', print_cmd=print_cmd)


In [ ]:
import os
import glob
from astropy.io import fits
from astropy.time import Time

# ----------------------------------
# INPUT
FIN_DIR = '/user/lprichard/project/sci_data/raw_sci/PID12345_rawdata_aquery2019Sep13/final'
# ----------------------------------

# Move into directory
os.chdir(FIN_DIR)

# Set time now
now = Time(Time.now(), format='iso')

# List flcs
flcs = glob.glob('*flc.fits')

# update filenames, history, date
n=0
for f in flcs:
    print('filename: {}'.format(f))

    # Open file for editing
    h = fits.open(f)

    # Update header
    h[0].header['FILENAME'] = '{}'.format(f)
    h[0].header['HISTORY'] = 'FLC corrections by L Prichard performed {}'.format(now)
    h[0].header['HISTORY'] = 'Code: https://github.com/lprichard/hst_wfc3_uvis_reduction.git'
    h[0].header['HISTORY'] = 'Includes median amp subtraction on FLCs by B Sunnquist'
    h[0].header['HISTORY'] = 'Code: https://github.com/bsunnquist/uvis-skydarks/blob/master/make_uvis_skydark.py'
    h[0].header['HISTORY'] = 'Includes read out cosmic rays (ROCRs) correction by B Sunnquist'
    h[0].header['HISTORY'] = 'Code: https://github.com/bsunnquist/uvis-skydarks/blob/master/flag_rocrs.ipynb'

    # Save file
    h.writeto(f, overwrite=True)
    h.close()
    n+=1
    print('Updated header for {}/{}: {}'.format(n, len(flcs), f))

print('Updated headers for {} final flcs in {}'.format(n, FIN_DIR))